In [1]:
from IPython.display import HTML
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import precision_score

import time
    
%matplotlib inline
import seaborn as sns

In [13]:
#Time Series Data
UR = pd.read_csv('UnempRate.csv', sep=',', na_values=[" ", ""])
crime = pd.read_csv('violent_crime_count_age17andup.csv', sep=',', na_values=[" ", ""])
#crime.iloc[:,1:]/population.iloc[:,1:] 
#but missing 2016 data, so 2016 data not used
#prescription is per 100
prescription = pd.read_csv('prescription_rate.csv', sep=',', na_values=[" ", ""])
population = pd.read_csv('population.csv', sep=',', na_values=[" ", ""])
birth_rate = pd.read_csv('birth_rate.csv', sep=',', na_values=[" ", ""])
death_rate = pd.read_csv('death_rate.csv', sep=',', na_values=[" ", ""])
children_poverty = pd.read_csv('children_living_in_poverty.csv', sep=',', na_values=[" ", ""])
teenage_pregnancy = pd.read_csv('Birth_Rate_10-19_Years_of_Age.csv', sep=',', na_values=[" ", ""])

In [4]:
#Normalizing Time-Series Data
UR.iloc[:,1:]=UR.iloc[:,1:]/100
birth_rate.iloc[:,1:]=birth_rate.iloc[:,1:]/1000
death_rate.iloc[:,1:]=death_rate.iloc[:,1:]/1000
prescription.iloc[:,1:]=prescription.iloc[:,1:]/100

In [5]:
#Non-Time Series Data
data = pd.read_csv('cleaned_GaCountiesODDeath.csv', na_values=[" ", ""] ,sep=',', index_col=0).sort('County').reset_index(drop=True)
hospital = pd.read_csv('cleaned_GaHospitals.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)
dropbox = pd.read_csv('drug_dropbox.csv', na_values=[" ", ""] ,sep=',').fillna(0)
ems = pd.read_csv('cleaned_ems.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)
rehab = pd.read_csv('cleaned_rehab.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)

hospital=hospital.rename(columns = {'Count':'hospital_count'})
dropbox=dropbox.rename(columns = {'COUNT':'dropbox_count'})
dropbox=dropbox.rename(columns = {'24Availability':'dropbox_247availability'})

In [6]:
normalized_data = data.iloc[:,1:]/population.iloc[:,1:]

In [7]:
df = data[['County','Rural']].copy()
df1 = pd.concat([df, rehab['Rehab_Count'],  ems['EMS_COUNT'],
                 hospital['hospital_count'], dropbox['dropbox_247availability'], dropbox['dropbox_count']], axis=1)

In [15]:
#data[['Counties','Rural', '2016', '1yrlag', '2yrlag', '10yrlag']]
df1['1yrlag']=(data['2016']-data['2015'])/data['2015']
df1['5yrlag']=(data['2016']-data['2011'])/data['2011']
df1['10yrlag']=(data['2016']-data['2006'])/data['2006']

df1['UR_1yrlag']=(UR['2016']-UR['2015'])/UR['2015']
df1['UR_5yrlag']=(UR['2016']-UR['2011'])/UR['2011']
df1['UR_10yrlag']=(UR['2016']-UR['2006'])/UR['2006']

df1['CR_1yrlag']=(crime['2015']-crime['2014'])/crime['2014']
df1['CR_5yrlag']=(crime['2015']-crime['2010'])/crime['2010']
df1['CR_10yrlag']=(crime['2015']-crime['2005'])/crime['2005']

df1['PR_1yrlag']=(prescription['2016']-prescription['2015'])/prescription['2015']
df1['PR_5yrlag']=(prescription['2016']-prescription['2011'])/prescription['2011']
df1['PR_10yrlag']=(prescription['2016']-prescription['2006'])/prescription['2006']

df1['BR_1yrlag']=(birth_rate['2016']-birth_rate['2015'])/birth_rate['2015']
df1['BR_5yrlag']=(birth_rate['2016']-birth_rate['2011'])/birth_rate['2011']
df1['BR_10yrlag']=(birth_rate['2016']-birth_rate['2006'])/birth_rate['2006']

df1['DR_1yrlag']=(death_rate['2016']-death_rate['2015'])/death_rate['2015']
df1['DR_5yrlag']=(death_rate['2016']-death_rate['2011'])/death_rate['2011']
df1['DR_10yrlag']=(death_rate['2016']-death_rate['2006'])/death_rate['2006']

df1=df1.replace([np.inf, -np.inf], np.nan)
df1 = df1.fillna(0)


teenage_pregnancy_2016=teenage_pregnancy.rename(columns = {'2016':'teenage_pregnancy_2016'})
children_poverty_2015=children_poverty.rename(columns = {'2015':'2015_children_poverty'})
death_rate_2016=death_rate.rename(columns = {'2016':'2016_death_rate'})
birth_rate_2016=birth_rate.rename(columns = {'2016':'2016_birth_rate'})
prescription_2016=prescription.rename(columns = {'2016':'2016_prescription'})

df2 = pd.concat([df1, teenage_pregnancy_2016['teenage_pregnancy_2016'], children_poverty_2015['2015_children_poverty'], death_rate_2016['2016_death_rate'], birth_rate_2016['2016_birth_rate'], prescription_2016['2016_prescription'], normalized_data['2016']], axis=1)
df2=df2.rename(columns = {'2016':'2016_ODRate'})

In [10]:
df2.columns
nat_OD_rate=0.00016219127

In [11]:
#2015 National OD Rate
#52,404/323100000
df2['2016ODabovenatavg'] = df2['2016_ODRate'] > nat_OD_rate

In [16]:
df2.to_csv('AggredgatedData.csv')